In [1]:
import re
import pandas as pd

In [2]:
### กำหนด data type ที่เหมาะสมกับ attribute values (Custom data types) ###


def guess_column_types(file_path, delimiter=',', has_headers=True):
    try:
        # Read the CSV file using the specified delimiter and header settings
        df = pd.read_csv(file_path, sep=delimiter,low_memory=False, header=0 if has_headers else None)

        # Initialize a dictionary to store column data types
        column_types = {}

        # Loop through columns and infer data types
        for column in df.columns:
            # sample_values = df[column].dropna().sample(min(5, len(df[column])), random_state=42)

            # Check for datetime format "YYYY-MM-DD HH:MM:SS"
            is_datetime = all(re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(value)) for value in df[column])

            # Check for date format "YYYY-MM-DD"
            is_date = all(re.match(r'\d{4}-\d{2}-\d{2}', str(value)) for value in df[column])

            # Assign data type based on format detection
            if is_datetime:
                inferred_type = 'datetime64'
            elif is_date:
                inferred_type = 'date'
            else:
                inferred_type = pd.api.types.infer_dtype(df[column], skipna=True)

            column_types[column] = inferred_type

        return (True, column_types)  # Return success and column types
    except pd.errors.ParserError:
        return (False, str(e))  # Return error message

In [3]:
file_path = './LoanStats_web.csv'
result, column_types_or_error = guess_column_types(file_path)

if result:
    print("Column Types:", column_types_or_error)
else:
    print("Error:", column_types_or_error)


Column Types: {'id': 'string', 'member_id': 'floating', 'loan_amnt': 'floating', 'funded_amnt': 'floating', 'funded_amnt_inv': 'floating', 'term': 'string', 'int_rate': 'string', 'installment': 'floating', 'grade': 'string', 'sub_grade': 'string', 'emp_title': 'string', 'emp_length': 'string', 'home_ownership': 'string', 'annual_inc': 'floating', 'verification_status': 'string', 'issue_d': 'string', 'loan_status': 'string', 'pymnt_plan': 'string', 'url': 'floating', 'desc': 'string', 'purpose': 'string', 'title': 'string', 'zip_code': 'string', 'addr_state': 'string', 'dti': 'floating', 'delinq_2yrs': 'floating', 'earliest_cr_line': 'string', 'inq_last_6mths': 'floating', 'mths_since_last_delinq': 'floating', 'mths_since_last_record': 'floating', 'open_acc': 'floating', 'pub_rec': 'floating', 'revol_bal': 'floating', 'revol_util': 'string', 'total_acc': 'floating', 'initial_list_status': 'string', 'out_prncp': 'floating', 'out_prncp_inv': 'floating', 'total_pymnt': 'floating', 'total_p

In [4]:
# เปลี่ยน data type บางตัว ให้เหมาะสมกับ python's environment และ MSSQL
column_types_corrected = {col: ('datetime64' if t == 'date' else 'float64' if t == 'floating' else t) \
                          for col, t in column_types_or_error.items()}

In [5]:
### นำ dictionary ที่บรรจุ ชื่อ col และ data type มาใช้เป็น parameter ของการทำ pd.read_csv ( ) อ่าน csv มาเป็น dataframe ###
raw_df = pd.read_csv(file_path, dtype=column_types_corrected, nrows=None).sample(frac=0.1, random_state=42)

In [6]:
# คำนวน percentage ของ missing values ของแต่ละ col. ใน dataframe (raw_df)
missing_percentage = raw_df.isnull().mean() * 100

# กรอง columns ที่มี null เกินกว่า 30% ออกไป
columns_to_keep = missing_percentage[missing_percentage <= 30].index.tolist()
filteredCol_df = raw_df[columns_to_keep]

In [7]:
raw_df = filteredCol_df

In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143247 entries, 976280 to 1361839
Data columns (total 100 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   loan_amnt                   143245 non-null  float64
 1   funded_amnt                 143245 non-null  float64
 2   funded_amnt_inv             143245 non-null  float64
 3   term                        143245 non-null  string 
 4   int_rate                    143245 non-null  string 
 5   installment                 143245 non-null  float64
 6   grade                       143245 non-null  string 
 7   sub_grade                   143245 non-null  string 
 8   emp_title                   130218 non-null  string 
 9   emp_length                  132384 non-null  string 
 10  home_ownership              143245 non-null  string 
 11  annual_inc                  143245 non-null  float64
 12  verification_status         143245 non-null  string 
 13  issue_d

In [9]:
raw_df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc

In [10]:
acceptableMax_null = 26

In [11]:
# สร้างรายการของคอลัมน์ที่มี Non-Null จำนวน 1,432,440
## selected_columns = [col for col in raw_df.columns if raw_df[col].notnull().sum() == 1432440]
selected_columns = [col for col in raw_df.columns if raw_df[col].isnull().sum() <= acceptableMax_null]

# แสดงคอลัมน์ที่เลือก
print("Selected columns:", selected_columns)

# สร้าง DataFrame ใหม่จากคอลัมน์ที่เลือก
df_selected = raw_df[selected_columns]


# ลบแถวที่มีค่า null ในคอลัมน์เหล่านั้น
noNull_df = df_selected.dropna()

# แสดงข้อมูลทั่วไปของ DataFrame หลังจากลบ null
noNull_df.info()

Selected columns: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'zip_code', 'addr_state', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_rev_tl_op', 'mo

In [12]:
# ทำสำเนาตัวแปร โดยหลังจากนี้ หากเกิดความเปลี่ยนแปลงกับ ตัวแปรใหม่ (df_prepared) จะไม่ส่งผลใดๆ ต่อตัวแปรเดิม (noNull_df)
df_prepared = noNull_df.copy()

# เปลี่ยน data type เป็น datetime สำหรับ col: issue_d
df_prepared['issue_d'] = pd.to_datetime(df_prepared['issue_d'], format='%b-%Y')

# นำเครื่องหมาย % ออกจากค่าใน col: int_rate แล้วเปลี่ยน data type เป็น float
if df_prepared['int_rate'].dtype == 'string':
    df_prepared['int_rate'] = df_prepared['int_rate'].str.rstrip('%').astype('float') / 100.0

In [13]:
df_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143217 entries, 976280 to 1361839
Data columns (total 85 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   loan_amnt                   143217 non-null  float64       
 1   funded_amnt                 143217 non-null  float64       
 2   funded_amnt_inv             143217 non-null  float64       
 3   term                        143217 non-null  string        
 4   int_rate                    143217 non-null  float64       
 5   installment                 143217 non-null  float64       
 6   grade                       143217 non-null  string        
 7   sub_grade                   143217 non-null  string        
 8   home_ownership              143217 non-null  string        
 9   annual_inc                  143217 non-null  float64       
 10  verification_status         143217 non-null  string        
 11  issue_d                     14321

In [14]:
df_prepared.groupby('issue_d').size()

issue_d
2016-01-01     3241
2016-02-01     3866
2016-03-01     6132
2016-04-01     3691
2016-05-01     2909
2016-06-01     3352
2016-07-01     3446
2016-08-01     3525
2016-09-01     2766
2016-10-01     3235
2016-11-01     3471
2016-12-01     3554
2017-01-01    12791
2017-02-01    11068
2017-03-01    14954
2018-01-01     3696
2018-02-01     3209
2018-03-01     3885
2018-04-01     4254
2018-05-01     4564
2018-06-01     4037
2018-07-01     4314
2018-08-01     4552
2018-09-01     3936
2018-10-01     4720
2018-11-01     4262
2018-12-01     4087
2019-01-01     4439
2019-02-01     3581
2019-03-01     3680
dtype: int64

In [15]:
df_prepared.groupby('int_rate').size()

int_rate
0.0531     861
0.0532    3803
0.0600      62
0.0607     527
0.0608     295
          ... 
0.3079     106
0.3084      70
0.3089      46
0.3094      29
0.3099      49
Length: 245, dtype: int64

In [16]:
## dimension table เป็นสิ่งที่อยู่ใน data warehouse ไม่ใช่ python
## แต่เราจะใช้ python ไป create dim. table ใน data warehouse พร้อม insert ข้อมูลลงไปด้วย
## ด้วยเหตุนี้ ครูเอ้จึงเสนอ idea สำหรับการใช้ python พัฒนา dim. table ดังนี้
## (1) สร้าง dataframe ขึ้นมาใหม่ เพื่อทำหน้าที่เป็น dim. table โดยมีเฉพาะ col. ตาม dimensional model ที่ออกแบบไว้แล้ว
## (2) สร้าง col. ใหม่ขึ้นมาบน dataframe ข้อ (1) เพื่อจะใช้เป็น primary key สำหรับ dim. table

# ทำข้อ (1) สำหรับ home_ownership
home_ownership_dim = df_prepared[['home_ownership']].drop_duplicates().reset_index(drop=True)
home_ownership_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ home_ownership
home_ownership_dim['home_ownership_id'] = home_ownership_dim.index

# ทำข้อ (1) สำหรับ loan_status
loan_status_dim = df_prepared[['loan_status']].drop_duplicates().reset_index(drop=True)
loan_status_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ loan_status
loan_status_dim['loan_status_id'] = loan_status_dim.index

# ทำข้อ (1) สำหรับ สำหรับ issue_d
issue_d_dim = df_prepared[['issue_d']].drop_duplicates().reset_index(drop=True)
issue_d_dim['month'] = issue_d_dim['issue_d'].dt.month
issue_d_dim['year'] = issue_d_dim['issue_d'].dt.year
issue_d_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ สำหรับ issue_d
issue_d_dim['issue_d_id'] = issue_d_dim.index

In [17]:
issue_d_dim

index    issue_d  month  year  issue_d_id
0       0 2018-05-01      5  2018           0
1       1 2016-05-01      5  2016           1
2       2 2016-02-01      2  2016           2
3       3 2016-08-01      8  2016           3
4       4 2016-03-01      3  2016           4
5       5 2017-03-01      3  2017           5
6       6 2018-02-01      2  2018           6
7       7 2019-02-01      2  2019           7
8       8 2016-10-01     10  2016           8
9       9 2018-10-01     10  2018           9
10     10 2016-09-01      9  2016          10
11     11 2016-07-01      7  2016          11
12     12 2018-12-01     12  2018          12
13     13 2018-03-01      3  2018          13
14     14 2018-11-01     11  2018          14
15     15 2016-06-01      6  2016          15
16     16 2017-02-01      2  2017          16
17     17 2018-04-01      4  2018          17
18     18 2018-08-01      8  2018          18
19     19 2018-06-01      6  2018          19
20     20 2018-09-01      9  2018          20
21     21 2018-07-01      7  2018          21
22     22 2016-04-01      4  2016          22
23     23 2016-01-01      1  2016          23
24     24 2016-12-01     12  2016          24
25     25 2017-01-01      1  2017          25
26     26 2019-01-01      1  2019          26
27     27 2019-03-01      3  2019          27
28     28 2016-11-01     11  2016          28
29     29 2018-01-01      1  2018          29

In [18]:
loan_status_dim

index         loan_status  loan_status_id
0      0             Current               0
1      1          Fully Paid               1
2      2         Charged Off               2
3      3   Late (16-30 days)               3
4      4  Late (31-120 days)               4
5      5     In Grace Period               5
6      6             Default               6

In [19]:
home_ownership_dim

index home_ownership  home_ownership_id
0      0           RENT                  0
1      1       MORTGAGE                  1
2      2            OWN                  2
3      3            ANY                  3
4      4           NONE                  4

In [20]:
home_ownership_dim.set_index('home_ownership')

index  home_ownership_id
home_ownership                          
RENT                0                  0
MORTGAGE            1                  1
OWN                 2                  2
ANY                 3                  3
NONE                4                  4

In [21]:
home_ownership_dim.set_index('home_ownership')['home_ownership_id']

home_ownership
RENT        0
MORTGAGE    1
OWN         2
ANY         3
NONE        4
Name: home_ownership_id, dtype: int64

In [22]:
home_ownership_dim.set_index('home_ownership')['home_ownership_id'].to_dict()

{'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'ANY': 3, 'NONE': 4}

In [23]:
loan_status_dim.set_index('loan_status')['loan_status_id'].to_dict()

{'Current': 0,
 'Fully Paid': 1,
 'Charged Off': 2,
 'Late (16-30 days)': 3,
 'Late (31-120 days)': 4,
 'In Grace Period': 5,
 'Default': 6}

In [24]:
issue_d_dim.set_index('issue_d')['issue_d_id'].to_dict()

{Timestamp('2018-05-01 00:00:00'): 0,
 Timestamp('2016-05-01 00:00:00'): 1,
 Timestamp('2016-02-01 00:00:00'): 2,
 Timestamp('2016-08-01 00:00:00'): 3,
 Timestamp('2016-03-01 00:00:00'): 4,
 Timestamp('2017-03-01 00:00:00'): 5,
 Timestamp('2018-02-01 00:00:00'): 6,
 Timestamp('2019-02-01 00:00:00'): 7,
 Timestamp('2016-10-01 00:00:00'): 8,
 Timestamp('2018-10-01 00:00:00'): 9,
 Timestamp('2016-09-01 00:00:00'): 10,
 Timestamp('2016-07-01 00:00:00'): 11,
 Timestamp('2018-12-01 00:00:00'): 12,
 Timestamp('2018-03-01 00:00:00'): 13,
 Timestamp('2018-11-01 00:00:00'): 14,
 Timestamp('2016-06-01 00:00:00'): 15,
 Timestamp('2017-02-01 00:00:00'): 16,
 Timestamp('2018-04-01 00:00:00'): 17,
 Timestamp('2018-08-01 00:00:00'): 18,
 Timestamp('2018-06-01 00:00:00'): 19,
 Timestamp('2018-09-01 00:00:00'): 20,
 Timestamp('2018-07-01 00:00:00'): 21,
 Timestamp('2016-04-01 00:00:00'): 22,
 Timestamp('2016-01-01 00:00:00'): 23,
 Timestamp('2016-12-01 00:00:00'): 24,
 Timestamp('2017-01-01 00:00:00'): 

In [25]:
## fact table เป็นสิ่งที่อยู่ใน data warehouse ไม่ใช่ python
## แต่เราจะใช้ python ไป create fact table ใน data warehouse พร้อม insert ข้อมูลลงไปด้วย
## ด้วยเหตุนี้ ครูเอ้จึงเสนอ idea สำหรับการใช้ python พัฒนา fact table ดังนี้
## (1) สร้าง python's dict. ขึ้นมาใช้ key mapping (สร้าง foreign key ของ fact table ตรงกับ primary key ของ dim. table)
## (2) dataframe ขึ้นมาใหม่ เพื่อทำหน้าที่เป็น fact table โดยนำ key mapping ตามข้อ 1 มาใช้ด้วย

In [26]:
# ทำข้อ (1)
home_ownership_map = home_ownership_dim.set_index('home_ownership')['home_ownership_id'].to_dict()
loan_status_map = loan_status_dim.set_index('loan_status')['loan_status_id'].to_dict()
issue_d_map = issue_d_dim.set_index('issue_d')['issue_d_id'].to_dict()

In [27]:
# ทำข้อ (2)
loans_fact = df_prepared.copy()
loans_fact['home_ownership_id'] = loans_fact['home_ownership'].map(home_ownership_map)
loans_fact['loan_status_id'] = loans_fact['loan_status'].map(loan_status_map)
loans_fact['issue_d_id'] = loans_fact['issue_d'].map(issue_d_map)

# เลือกคอลัมน์ที่จำเป็นสำหรับ Fact Table
loans_fact = loans_fact[['application_type','loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment'\
                         , 'home_ownership_id', 'loan_status_id', 'issue_d_id']]

In [28]:
loans_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143217 entries, 976280 to 1361839
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   application_type   143217 non-null  string 
 1   loan_amnt          143217 non-null  float64
 2   funded_amnt        143217 non-null  float64
 3   term               143217 non-null  string 
 4   int_rate           143217 non-null  float64
 5   installment        143217 non-null  float64
 6   home_ownership_id  143217 non-null  int64  
 7   loan_status_id     143217 non-null  int64  
 8   issue_d_id         143217 non-null  int64  
dtypes: float64(4), int64(3), string(2)
memory usage: 10.9 MB


In [29]:
### เริ่มต้นเปลี่ยนเป็น raw cell ชั่วคราว ###

In [30]:
## [optional] ทดสอบการ join ทุก dataframe ทั้ง fact และ dim. เข้าด้วยกัน

# Join `loans_fact` กับ `home_ownership_dim` โดยใช้ 'home_ownership_id'
loans_fact_with_home_ownership = pd.merge(loans_fact, home_ownership_dim, on='home_ownership_id', how='left', suffixes=('', '_home_ownership'))

# Join ผลลัพธ์กับ `loan_status_dim` โดยใช้ 'loan_status_id'
loans_fact_with_home_ownership_with_loan_status = pd.merge(loans_fact_with_home_ownership, loan_status_dim, on='loan_status_id', how='left', suffixes=('', '_loan_status'))

# Join ผลลัพธ์กับ `issue_d_dim` โดยใช้ 'issue_d_id'
final_df = pd.merge(loans_fact_with_home_ownership_with_loan_status, issue_d_dim, on='issue_d_id', how='left', suffixes=('', '_issue_d'))


In [31]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องมี row เท่ากันกับ dataframe เดิม
print(f"จำนวนแถวใน noNull_df: {noNull_df.shape[0]}")
print(f"จำนวนแถวใน final_df: {final_df.shape[0]}")

จำนวนแถวใน noNull_df: 143217
จำนวนแถวใน final_df: 143217


In [32]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องมีค่าของ measure เท่ากันกับ dataframe เดิม
print("การเปรียบเทียบค่า loan_amnt จาก dataframe เดิม กับ fact:")
print(noNull_df['funded_amnt'].head())
print(final_df['funded_amnt'].head())


การเปรียบเทียบค่า loan_amnt จาก dataframe เดิม กับ fact:
976280    33600.0
180653     9600.0
67613     10400.0
282861    10000.0
21432     24000.0
Name: funded_amnt, dtype: float64
0    33600.0
1     9600.0
2    10400.0
3    10000.0
4    24000.0
Name: funded_amnt, dtype: float64


In [33]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องไม่มี NULL เลย
print("จำนวนค่า Null ใน final_df หลังจากการ join:")
print(final_df.isnull().sum())


จำนวนค่า Null ใน final_df หลังจากการ join:
application_type     0
loan_amnt            0
funded_amnt          0
term                 0
int_rate             0
installment          0
home_ownership_id    0
loan_status_id       0
issue_d_id           0
index                0
home_ownership       0
index_loan_status    0
loan_status          0
index_issue_d        0
issue_d              0
month                0
year                 0
dtype: int64


In [34]:
### สิ้นสุดเปลี่ยนเป็น raw cell ชั่วคราว ###

In [35]:
issue_d_dim.drop(columns=['index'], inplace=True)

In [36]:
loan_status_dim.drop(columns=['index'], inplace=True)

In [37]:
home_ownership_dim.drop(columns=['index'], inplace=True)

In [38]:
server = '35.224.12.16'
database = 'TestDB'
username = 'SA'
password = 'Passw0rd123456'

In [39]:
from sqlalchemy import create_engine
import urllib

# ตั้งค่าการเชื่อมต่อกับ MSSQL โดยใช้ข้อมูลที่เหมาะสมกับสภาพแวดล้อมของคุณ
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=server;DATABASE=database;UID=username;PWD=password")
##### engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
# Using pymssql
engine = create_engine(f'mssql+pymssql://{username}:{password}@{server}/{database}')


# นำเข้าข้อมูล Dimension Tables ไปยัง MSSQL ให้ตรงกับขั้นตอนการเตรียมข้อมูลที่คุณทำไว้
##### home_ownership_dim.to_sql('home_ownership_dim', con=engine, if_exists='replace', index=True, index_label='home_ownership_id')
##### loan_status_dim.to_sql('loan_status_dim', con=engine, if_exists='replace', index=True, index_label='loan_status_id')

home_ownership_dim.to_sql('home_ownership_dim', con=engine, if_exists='replace', index=False)
loan_status_dim.to_sql('loan_status_dim', con=engine, if_exists='replace', index=False)
issue_d_dim.to_sql('issue_d_dim', con=engine, if_exists='replace', index=False)



# นำเข้าข้อมูล Fact Table ไปยัง MSSQL โดยใช้ข้อมูลที่เตรียมไว้
loans_fact.to_sql('loans_fact', con=engine, if_exists='replace', index=False)


155